In [84]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pygame
import sys

In [92]:
# Genrate Medications
meds_df = pd.DataFrame({
    'Medicine': [],
    'Lifespan': []
})
max_meds = 10
lifespan_mean = 25
lifespan_std = 10

LIFESPAN_MIN = 5

medicine = set(['{:04x}'.format(np.random.randint(0, 0xFFFF)) for _ in range(max_meds)])
# ensures there are max_meds unique values
while len(medicine) < max_meds: medicine.add('{:04x}'.format(np.random.randint(0, 0xFFFF)))
medicine = list(medicine)

lifespan = np.random.normal(lifespan_mean, lifespan_std, max_meds)
lifespan = np.round(np.maximum(lifespan, LIFESPAN_MIN))

meds_df['Medicine'] = medicine
meds_df['Lifespan'] = lifespan

meds_df.head()


Medicine  Lifespan
0     6525      16.0
1     1b88      29.0
2     6677      26.0
3     4956      37.0
4     70de       5.0

In [93]:
# Generate Clients
client_count = 100
MAXLAT = 256
MAXLONG = 256
MAX_DOSAGE = 3

latitudes = np.random.randint(0, MAXLAT, client_count)
longitudes = np.random.randint(0, MAXLONG, client_count)

cs_df = pd.DataFrame({
    'Client': ['Client {:03d}'.format(i) for i in range(client_count)],
    'Lat': latitudes,
    'Long': longitudes,
    'Medicine': np.random.choice(medicine, client_count),
    'Dosage': np.random.randint(1, MAX_DOSAGE, client_count)
})
# cs_df.index.name = 'Client'
cs_df.head()

Client  Lat  Long Medicine  Dosage
0  Client 000  180    94     2e4e       1
1  Client 001  102    45     330b       1
2  Client 002  113   237     6525       1
3  Client 003   85    69     4956       1
4  Client 004  135    20     1b88       1

In [94]:
# Generate Pharmacies
pharmacy_count = 10
pharmacy_names = [f'pharm{i}' for i in range(pharmacy_count)]

latitudes = np.random.randint(0, MAXLAT, pharmacy_count)
longitudes = np.random.randint(0, MAXLONG, pharmacy_count)

rx_df = pd.DataFrame({
    'Name': pharmacy_names,
    'Lat': latitudes,
    'Long': longitudes
})
rx_df

Name  Lat  Long
0  pharm0  128    36
1  pharm1  161    30
2  pharm2   99    95
3  pharm3   88   236
4  pharm4  113   245
5  pharm5  114   232
6  pharm6   57    56
7  pharm7   91    29
8  pharm8  248   209
9  pharm9  191    15

In [95]:
# Assign Customers to Pharmacies
# client are assigned to closest pharm

# distance between each client and each pharm
client_coords = cs_df[['Lat', 'Long']].values
pharm = np.column_stack((latitudes[:pharmacy_count], longitudes[:pharmacy_count]))

distances = np.sqrt(
    (
        (client_coords[:, np.newaxis, :] - pharm[np.newaxis, :, :]) ** 2
    ).sum(axis=2))


# Assign each customer to the closest pharmacy
closest_pharmacies = np.argmin(distances, axis=1)
cs_df['Assigned_Pharmacy'] = closest_pharmacies

cs_df.head()


Client  Lat  Long Medicine  Dosage  Assigned_Pharmacy
0  Client 000  180    94     2e4e       1                  1
1  Client 001  102    45     330b       1                  7
2  Client 002  113   237     6525       1                  5
3  Client 003   85    69     4956       1                  2
4  Client 004  135    20     1b88       1                  0

In [99]:
# Sort by assigned pharmacy
total_cs_df = cs_df.groupby(['Assigned_Pharmacy','Medicine']).sum().reset_index().set_index('Assigned_Pharmacy')
total_cs_df = total_cs_df.drop(columns='Client')
total_cs_df

Medicine   Lat  Long  Dosage
Assigned_Pharmacy                             
0                     09bc   122    20       2
0                     1b88   264    91       3
0                     7476   132    37       1
1                     1b88   189    59       1
1                     2e4e   368   210       2
1                     6677   153    59       2
1                     7476   384   158       3
1                     a5dc   156    26       1
2                     09bc   196   242       4
2                     1b88   155   148       1
2                     330b   169   119       1
2                     4956   441   559       6
2                     6525   277   267       2
2                     6677   109   124       2
2                     70de   497   449       6
2                     7476   297   383       4
2                     a5dc   131    85       2
3                     09bc   100   419       4
3                     1b88     4   163       2
3                     4956   159   629       5
3                     70de    56   203       2
3                     7476    17   224       1
5                     09bc   414   553       5
5                     2e4e   143   196       1
5                     4956    97   172       1
5                     6525   371   594       4
5                     6677   176   196       2
5                     7476   135   199       2
5                     a5dc   109   214       1
6                     09bc    71   178       4
6                     1b88    52    33       1
6                     2e4e    67    64       2
6                     4956    25   219       4
6                     6525    23   111       2
6                     6677    71   155       3
6                     70de    83   104       3
6                     a5dc     5   190       4
7                     330b   102    45       1
7                     6525   207    76       4
7                     70de   172    62       3
8                     09bc   724   537       4
8                     1b88   243   174       1
8                     2e4e   439   415       3
8                     330b   432   471       4
8                     4956  1164   905       6
8                     6525   231   199       2
8                     6677   224   223       2
8                     70de   910   621       8
8                     a5dc   253   188       2
9                     09bc   234    57       2
9                     1b88   201    17       1
9                     2e4e   198    29       2
9                     4956   170     0       2
9                     6525   408    74       3
9                     6677   194     6       2
9                     70de   215    96       1
9                     7476   643   201       4

In [110]:
merged_df = pd.merge(rx_df, total_cs_df, right_index=True, left_index=True, how='left')

In [111]:
# Assign each pharmacy a list of medications that are required
from collections import Counter

# use assigned_pharmacy in cs_df to get the pharmacy name, and then get the medicine
# turn that into a counter and then into a dataframe into demand
def calculate_demand(group):
    return Counter({medicine: group.loc[group['Medicine'] == medicine, 'Dosage'].sum() for medicine in group['Medicine'].unique()})

# Apply the function to group by pharmacy ('Name') and calculate demand
demand_agg = merged_df.groupby('Name').apply(calculate_demand)

# Assign the aggregated demand back to the original DataFrame
pharmacy_df = merged_df.drop_duplicates(subset='Name').set_index('Name')
pharmacy_df['Demand'] = demand_agg

# Display the result
pharmacy_df.reset_index()


/tmp/ipykernel_48747/1434849096.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  demand_agg = merged_df.groupby('Name').apply(calculate_demand)


Name  Lat_x  Long_x Medicine  Lat_y  Long_y  Dosage  \
0  pharm0    128      36     09bc  122.0    20.0     2.0   
1  pharm1    161      30     1b88  189.0    59.0     1.0   
2  pharm2     99      95     09bc  196.0   242.0     4.0   
3  pharm3     88     236     09bc  100.0   419.0     4.0   
4  pharm4    113     245      NaN    NaN     NaN     NaN   
5  pharm5    114     232     09bc  414.0   553.0     5.0   
6  pharm6     57      56     09bc   71.0   178.0     4.0   
7  pharm7     91      29     330b  102.0    45.0     1.0   
8  pharm8    248     209     09bc  724.0   537.0     4.0   
9  pharm9    191      15     09bc  234.0    57.0     2.0   

                                              Demand  
0            {'09bc': 2.0, '1b88': 3.0, '7476': 1.0}  
1  {'1b88': 1.0, '2e4e': 2.0, '6677': 2.0, '7476'...  
2  {'09bc': 4.0, '1b88': 1.0, '330b': 1.0, '4956'...  
3  {'09bc': 4.0, '1b88': 2.0, '4956': 5.0, '70de'...  
4                                         {nan: 0.0}  
5  {'09bc': 5.0, '2e4e': 1.0, '4956': 1.0, '6525'...  
6  {'09bc': 4.0, '1b88': 1.0, '2e4e': 2.0, '4956'...  
7            {'330b': 1.0, '6525': 4.0, '70de': 3.0}  
8  {'09bc': 4.0, '1b88': 1.0, '2e4e': 3.0, '330b'...  
9  {'09bc': 2.0, '1b88': 1.0, '2e4e': 2.0, '4956'...